In [ ]:
from pathlib import Path
import pandas as pd
from collections import OrderedDict

# Raíz del repo
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
df = pd.read_csv(ROOT / "data" / "raw" / "Tab_Clientes(2).csv", low_memory=False)

# Parse mínimos
f_ult = pd.to_datetime(df.get("FecUltimoPptoCreado"), errors="coerce", dayfirst=True)

def resumen(tag, d: pd.DataFrame):
    out = OrderedDict()
    out["tag"] = tag
    out["n_pacientes"] = len(d)
    out["top_empresas"] = d.get("Empresa", pd.Series(dtype=str)).astype(str).str.upper().value_counts().head(10).to_dict()
    out["top_comunas"]  = d.get("Comuna", pd.Series(dtype=str)).astype(str).value_counts().head(10).to_dict()
    out["rango_edad"]   = d.get("RangoEdad", pd.Series(dtype=str)).value_counts().head(10).to_dict()
    # Género (si existe)
    if "Sexo" in d.columns:
        out["genero"] = d["Sexo"].value_counts(dropna=False).to_dict()
    return out

# Corte A: "2025" aprox por última fecha de ppto (limitación: no es transaccional)
mask_2025 = f_ult.dt.year.eq(2025)
res_2025  = resumen("Corte A ~ Año 2025 (por FecUltimoPptoCreado)", df[mask_2025].copy())

# Corte B: activos (≤ 2 años) – lo que usamos para clustering
dias = pd.to_numeric(df.get("DiasDesdeUltimaVisita"), errors="coerce")
mask_act = dias.le(730)
res_act  = resumen("Corte B ~ Activos (≤ 2 años)", df[mask_act].copy())

for r in (res_2025, res_act):
    print("\n===", r["tag"], "===")
    print("n_pacientes:", r["n_pacientes"])
    print("Top empresas:", r["top_empresas"])
    print("Top comunas :", r["top_comunas"])
    print("Rango edad  :", r["rango_edad"])
    if "genero" in r:
        print("Género:", r["genero"])
